In [30]:
# from pymongo import MongoClient
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import LabelEncoder
# import numpy as np

# # Conexión a MongoDB
# client = MongoClient("mongodb://localhost:27017/")
# db = client["my_database"]
# orders_table = db["orders"]
# products_table = db["products"]

# # 1. Extraer datos de órdenes y productos
# def extract_data():
#     # Extraer órdenes
#     orders = list(orders_table.find({}, {"Customer ID": 1, "Product ID": 1, "_id": 0}))
#     orders_df = pd.DataFrame(orders)
    
#     # Extraer productos (incluyendo categoría y nombre)
#     products = list(products_table.find({}, {"Product ID": 1, "Category": 1, "Name": 1, "_id": 0}))
#     products_df = pd.DataFrame(products)
    
#     # Unir productos con categorías y nombres
#     orders_with_categories = orders_df.merge(products_df, on="Product ID")

#     # Dividir en entrenamiento y prueba
#     train_df = orders_with_categories.sample(frac=0.8, random_state=42)
#     test_df = orders_with_categories.drop(train_df.index)

#     return train_df, test_df

# # 2. Crear matriz de usuario-producto
# def create_user_product_matrix(orders_with_categories):
#     # Codificar IDs de usuarios y productos
#     user_encoder = LabelEncoder()
#     product_encoder = LabelEncoder()

#     orders_with_categories["User Index"] = user_encoder.fit_transform(orders_with_categories["Customer ID"])
#     orders_with_categories["Product Index"] = product_encoder.fit_transform(orders_with_categories["Product ID"])

#     # Crear matriz dispersa usuario-producto
#     num_users = orders_with_categories["User Index"].nunique()
#     num_products = orders_with_categories["Product Index"].nunique()
#     matrix = np.zeros((num_users, num_products))

#     for _, row in orders_with_categories.iterrows():
#         matrix[row["User Index"], row["Product Index"]] += 1  # Cuenta de interacciones

#     return matrix, user_encoder, product_encoder, orders_with_categories

# # 3. Recomendar productos usando similitud de coseno (con filtro de categorías)
# def recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=10):
#     # Verificar si el usuario existe
#     if user_id not in user_encoder.classes_:
#         print(f"El usuario {user_id} no tiene datos suficientes para generar recomendaciones.")
#         return []

#     user_index = user_encoder.transform([user_id])[0]

#     # Calcular similitud entre usuarios
#     user_similarities = cosine_similarity(matrix)
#     similar_users = np.argsort(-user_similarities[user_index])  # Ordenar usuarios similares

#     # Filtrar productos comprados por categorías del usuario
#     user_categories = orders_with_categories[orders_with_categories["Customer ID"] == user_id]["Category"].unique()
#     candidate_products = orders_with_categories[orders_with_categories["Category"].isin(user_categories)]["Product Index"].unique()

#     # Recomendar productos basados en usuarios similares
#     scores = matrix[similar_users[1:], :].sum(axis=0)  # Ignorar al propio usuario
#     scores = [(i, scores[i]) for i in candidate_products if scores[i] > 0]
#     scores = sorted(scores, key=lambda x: x[1], reverse=True)

#     # Obtener los productos recomendados (ID y nombres)
#     product_mapping = orders_with_categories[["Product Index", "Product ID", "Name"]].drop_duplicates()
#     product_mapping = product_mapping.set_index("Product Index")

#     recommended_products = [
#         (product_mapping.loc[product[0], "Product ID"], product_mapping.loc[product[0], "Name"], product[1])
#         for product in scores[:num_recommendations]
#     ]
#     return recommended_products

# # 4. Evaluar el modelo
# def evaluate_model(test_df, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=5):
#     precision_scores = []
#     recall_scores = []
#     all_recommendations = {}

#     for user_id in test_df["Customer ID"].unique():
#         # Verificar si el usuario está en el conjunto de entrenamiento
#         if user_id not in user_encoder.classes_:
#             continue

#         # Productos comprados en el conjunto de prueba
#         relevant_products = test_df[test_df["Customer ID"] == user_id]["Product ID"].unique()

#         # Obtener recomendaciones
#         recommendations = recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations)
#         recommended_products = [rec[0] for rec in recommendations]  # IDs de productos recomendados

#         # Guardar recomendaciones por usuario
#         all_recommendations[user_id] = recommendations

#         # Calcular métricas
#         relevant_recommended = set(recommended_products) & set(relevant_products)
#         precision = len(relevant_recommended) / num_recommendations
#         recall = len(relevant_recommended) / len(relevant_products) if len(relevant_products) > 0 else 0

#         precision_scores.append(precision)
#         recall_scores.append(recall)

#     # Promediar métricas
#     avg_precision = np.mean(precision_scores)
#     avg_recall = np.mean(recall_scores)

#     return avg_precision, avg_recall, all_recommendations

# # Ejecutar el sistema de recomendación
# if __name__ == "__main__":
#     # Extraer datos y construir la matriz usuario-producto
#     train_df, test_df = extract_data()
#     matrix, user_encoder, product_encoder, orders_with_categories = create_user_product_matrix(train_df)

#     # Evaluar el modelo
#     avg_precision, avg_recall, all_recommendations = evaluate_model(test_df, matrix, user_encoder, product_encoder, train_df, num_recommendations=5)
#     print(f"Precision@5: {avg_precision:.2f}, Recall@5: {avg_recall:.2f}")

#     # Mostrar recomendaciones para algunos usuarios
#     for user_id, recommendations in list(all_recommendations.items())[:1]:  # Mostrar para un usuario
#         print(f"Recomendaciones para el usuario {user_id}: {recommendations}")


Precision@5: 0.93, Recall@5: 0.10
Recomendaciones para el usuario 1f63eb5a-531d-4398-85c4-643deb6804de: [('0aff57c9-2036-4878-b4d1-eec4ac524757', 'Blue Dot Rolling Pizza Cutter', np.float64(10384.0)), ('98f4bdac-73c8-424e-9b96-efe0844aefe1', 'Jhondeal.com Wheel Pizza Cutter', np.float64(10282.0)), ('662d3b8f-ba4d-459f-9cd9-91ac837503c8', 'Mom Italy Wheel Pizza Cutter', np.float64(10273.0)), ('fa5ab25d-8ece-4b0f-b81a-1f1d0c64a46b', 'Made In China Rolling Pizza Cutter', np.float64(10264.0)), ('21c56d5f-f241-421f-8bad-c1938138ef02', 'Apex Rolling Pizza Cutter', np.float64(10207.0))]


# Codigo anterior con mejoras / Relevancia normalizada

In [38]:
# from pymongo import MongoClient
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import LabelEncoder
# import numpy as np

# # Conexión a MongoDB
# client = MongoClient("mongodb://localhost:27017/")
# db = client["my_database"]
# orders_table = db["orders"]
# products_table = db["products"]

# # 1. Extraer datos de órdenes y productos
# def extract_data():
#     orders = list(orders_table.find({}, {"Customer ID": 1, "Product ID": 1, "_id": 0}))
#     orders_df = pd.DataFrame(orders)

#     products = list(products_table.find({}, {"Product ID": 1, "Category": 1, "Name": 1, "_id": 0}))
#     products_df = pd.DataFrame(products)

#     orders_with_categories = orders_df.merge(products_df, on="Product ID")
#     train_df = orders_with_categories.sample(frac=0.8, random_state=42)
#     test_df = orders_with_categories.drop(train_df.index)

#     return train_df, test_df

# # 2. Crear matriz de usuario-producto
# def create_user_product_matrix(orders_with_categories):
#     user_encoder = LabelEncoder()
#     product_encoder = LabelEncoder()

#     orders_with_categories["User Index"] = user_encoder.fit_transform(orders_with_categories["Customer ID"])
#     orders_with_categories["Product Index"] = product_encoder.fit_transform(orders_with_categories["Product ID"])

#     num_users = orders_with_categories["User Index"].nunique()
#     num_products = orders_with_categories["Product Index"].nunique()
#     matrix = np.zeros((num_users, num_products))

#     for _, row in orders_with_categories.iterrows():
#         matrix[row["User Index"], row["Product Index"]] += 1

#     return matrix, user_encoder, product_encoder, orders_with_categories

# # 3. Recomendar productos usando similitud de coseno
# def recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=10):
#     if user_id not in user_encoder.classes_:
#         print(f"El usuario {user_id} no tiene datos suficientes para generar recomendaciones.")
#         return [], []

#     user_index = user_encoder.transform([user_id])[0]
#     user_similarities = cosine_similarity(matrix)
#     similar_users = np.argsort(-user_similarities[user_index])

#     user_categories = orders_with_categories[orders_with_categories["Customer ID"] == user_id]["Category"].unique()
#     candidate_products = orders_with_categories[orders_with_categories["Category"].isin(user_categories)]["Product Index"].unique()

#     scores = matrix[similar_users[1:], :].sum(axis=0)
#     max_score = scores.max() if scores.max() > 0 else 1  # Evitar división por 0
#     normalized_scores = [(i, scores[i] / max_score) for i in candidate_products if scores[i] > 0]
#     normalized_scores = sorted(normalized_scores, key=lambda x: x[1], reverse=True)

#     product_mapping = orders_with_categories[["Product Index", "Product ID", "Name"]].drop_duplicates()
#     product_mapping = product_mapping.set_index("Product Index")

#     recommended_products = [
#         (product_mapping.loc[product[0], "Product ID"], product_mapping.loc[product[0], "Name"], round(product[1], 2))
#         for product in normalized_scores[:num_recommendations]
#     ]

#     user_purchases = orders_with_categories[orders_with_categories["Customer ID"] == user_id][["Product ID", "Name", "Category"]].drop_duplicates()

#     return recommended_products, user_purchases

# # 4. Evaluar el modelo
# def evaluate_model(test_df, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=5):
#     precision_scores = []
#     recall_scores = []
#     all_recommendations = {}

#     for user_id in test_df["Customer ID"].unique():
#         if user_id not in user_encoder.classes_:
#             continue

#         relevant_products = test_df[test_df["Customer ID"] == user_id]["Product ID"].unique()

#         recommendations, _ = recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations)
#         recommended_products = [rec[0] for rec in recommendations]

#         all_recommendations[user_id] = recommendations

#         relevant_recommended = set(recommended_products) & set(relevant_products)
#         precision = len(relevant_recommended) / num_recommendations
#         recall = len(relevant_recommended) / len(relevant_products) if len(relevant_products) > 0 else 0

#         precision_scores.append(precision)
#         recall_scores.append(recall)

#     avg_precision = np.mean(precision_scores)
#     avg_recall = np.mean(recall_scores)

#     return avg_precision, avg_recall, all_recommendations

# # Ejecutar el sistema de recomendación
# if __name__ == "__main__":
#     train_df, test_df = extract_data()
#     matrix, user_encoder, product_encoder, orders_with_categories = create_user_product_matrix(train_df)

#     avg_precision, avg_recall, all_recommendations = evaluate_model(test_df, matrix, user_encoder, product_encoder, train_df, num_recommendations=5)
#     print(f"Precision@5: {avg_precision:.2f}, Recall@5: {avg_recall:.2f}")

#     for user_id in test_df["Customer ID"].unique()[:1]:
#         recommendations, user_purchases = recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=5)
#         print(f"\nRecomendaciones para el usuario {user_id}:")
#         print(pd.DataFrame(recommendations, columns=["Product ID", "Name", "Relevancia"]))
#         print("-"*50)
#         print(f"\nProductos comprados por el usuario {user_id}:")
#         print(user_purchases)


Precision@5: 0.27, Recall@5: 0.16

Recomendaciones para el usuario 09ea6f56-580d-472a-b9e0-803b4c8583f3:
                             Product ID  \
0  47f97163-3d02-49b3-896a-2632ccc811b7   
1  b91a34eb-3530-442f-83c3-808dceb4b2af   
2  af4f968c-2dfd-43af-b15c-2c824c01a775   
3  07c70418-9916-46a4-96e0-dc821d034856   
4  3b65dc65-2e8e-4988-91d5-e56288de12af   

                                     Name  Relevancia  
0   Blue Heaven Eyeliner (Set of 3) 21 ml        0.54  
1  Organistick Silver Label Lipstick 10 g        0.53  
2      Incolor Metalic Lipstick N15 3.8 g        0.47  
3            Miss Lips Lipstick - 152 4 g        0.43  
4            Miss Lips Lipstick - 138 4 g        0.42  
--------------------------------------------------

Productos comprados por el usuario 09ea6f56-580d-472a-b9e0-803b4c8583f3:
                                  Product ID  \
121765  8da6f578-49cd-4deb-ba1a-fb2594a7ae88   
114290  cbec1619-2653-4f67-991b-13f2fae38306   
33544   486b70ce-cc11-420c-b18b

# Intento de mejora del anterior teniendo en cuenta lo que ha comprado el usuario

In [35]:
# from pymongo import MongoClient
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import LabelEncoder
# import numpy as np

# # Conexión a MongoDB
# client = MongoClient("mongodb://localhost:27017/")
# db = client["my_database"]
# orders_table = db["orders"]
# products_table = db["products"]

# # 1. Extraer datos de órdenes y productos
# def extract_data():
#     orders = list(orders_table.find({}, {"Customer ID": 1, "Product ID": 1, "_id": 0}))
#     orders_df = pd.DataFrame(orders)

#     products = list(products_table.find({}, {"Product ID": 1, "Category": 1, "Name": 1, "_id": 0}))
#     products_df = pd.DataFrame(products)

#     orders_with_categories = orders_df.merge(products_df, on="Product ID")
#     train_df = orders_with_categories.sample(frac=0.8, random_state=42)
#     test_df = orders_with_categories.drop(train_df.index)

#     return train_df, test_df

# # 2. Crear matriz de usuario-producto
# def create_user_product_matrix(orders_with_categories):
#     user_encoder = LabelEncoder()
#     product_encoder = LabelEncoder()

#     orders_with_categories["User Index"] = user_encoder.fit_transform(orders_with_categories["Customer ID"])
#     orders_with_categories["Product Index"] = product_encoder.fit_transform(orders_with_categories["Product ID"])

#     num_users = orders_with_categories["User Index"].nunique()
#     num_products = orders_with_categories["Product Index"].nunique()
#     matrix = np.zeros((num_users, num_products))

#     for _, row in orders_with_categories.iterrows():
#         matrix[row["User Index"], row["Product Index"]] += 1

#     return matrix, user_encoder, product_encoder, orders_with_categories

# # 3. Recomendar productos usando similitud de coseno
# def recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=5):
#     if user_id not in user_encoder.classes_:
#         print(f"El usuario {user_id} no tiene datos suficientes para generar recomendaciones.")
#         return [], []

#     user_index = user_encoder.transform([user_id])[0]
#     user_similarities = cosine_similarity(matrix)
#     similar_users = np.argsort(-user_similarities[user_index])

#     user_categories = orders_with_categories[orders_with_categories["Customer ID"] == user_id]["Category"].unique()
#     candidate_products = orders_with_categories[orders_with_categories["Category"].isin(user_categories)]["Product Index"].unique()

#     # Excluir productos ya comprados por el usuario
#     purchased_products = orders_with_categories[orders_with_categories["Customer ID"] == user_id]["Product Index"].unique()
#     candidate_products = [product for product in candidate_products if product not in purchased_products]

#     # Recomendar productos basados en usuarios similares
#     scores = matrix[similar_users[1:], :].sum(axis=0)
#     max_score = scores.max() if scores.max() > 0 else 1
#     normalized_scores = [(i, scores[i] / max_score) for i in candidate_products if scores[i] > 0]
#     normalized_scores = sorted(normalized_scores, key=lambda x: x[1], reverse=True)

#     product_mapping = orders_with_categories[["Product Index", "Product ID", "Name"]].drop_duplicates()
#     product_mapping = product_mapping.set_index("Product Index")

#     recommended_products = [
#         (product_mapping.loc[product[0], "Product ID"], product_mapping.loc[product[0], "Name"], round(product[1], 2))
#         for product in normalized_scores[:num_recommendations]
#     ]

#     user_purchases = orders_with_categories[orders_with_categories["Customer ID"] == user_id][["Product ID", "Name", "Category"]].drop_duplicates()

#     return recommended_products, user_purchases

# # 4. Evaluar el modelo
# def evaluate_model(test_df, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=10):
#     precision_scores = []
#     recall_scores = []
#     all_recommendations = {}

#     for user_id in test_df["Customer ID"].unique():
#         if user_id not in user_encoder.classes_:
#             continue

#         relevant_products = test_df[test_df["Customer ID"] == user_id]["Product ID"].unique()

#         recommendations, _ = recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations)
#         recommended_products = [rec[0] for rec in recommendations]

#         all_recommendations[user_id] = recommendations

#         relevant_recommended = set(recommended_products) & set(relevant_products)
#         precision = len(relevant_recommended) / num_recommendations
#         recall = len(relevant_recommended) / len(relevant_products) if len(relevant_products) > 0 else 0

#         precision_scores.append(precision)
#         recall_scores.append(recall)

#     avg_precision = np.mean(precision_scores)
#     avg_recall = np.mean(recall_scores)

#     return avg_precision, avg_recall, all_recommendations

# # Ejecutar el sistema de recomendación
# if __name__ == "__main__":
#     train_df, test_df = extract_data()
#     matrix, user_encoder, product_encoder, orders_with_categories = create_user_product_matrix(train_df)

#     avg_precision, avg_recall, all_recommendations = evaluate_model(test_df, matrix, user_encoder, product_encoder, train_df, num_recommendations=10)
#     print(f"Precision@10: {avg_precision:.2f}, Recall@10: {avg_recall:.2f}")

#     for user_id in test_df["Customer ID"].unique()[:1]:
#         recommendations, user_purchases = recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=10)
#         print(f"\nRecomendaciones para el usuario {user_id}:")
#         print(pd.DataFrame(recommendations, columns=["Product ID", "Name", "Relevancia"]))
#         print("-"*50)
#         print(f"\nProductos comprados por el usuario {user_id}:")
#         print(user_purchases)


KeyboardInterrupt: 

In [2]:
from pymongo import MongoClient
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle

# Conexión a MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"]
orders_table = db["orders"]
products_table = db["products"]

# 1. Extraer datos de órdenes y productos con filtrado
def extract_data(min_orders_per_user=5, min_users_per_product=5):
    orders = list(orders_table.find({}, {"Customer ID": 1, "Product ID": 1, "_id": 0}))
    orders_df = pd.DataFrame(orders)

    products = list(products_table.find({}, {"Product ID": 1, "Category": 1, "Name": 1, "_id": 0}))
    products_df = pd.DataFrame(products)

    orders_with_categories = orders_df.merge(products_df, on="Product ID")

    # Filtrar usuarios con pocas órdenes
    user_order_counts = orders_with_categories.groupby("Customer ID").size()
    valid_users = user_order_counts[user_order_counts >= min_orders_per_user].index
    orders_with_categories = orders_with_categories[orders_with_categories["Customer ID"].isin(valid_users)]

    # Filtrar productos con pocas compras
    product_order_counts = orders_with_categories.groupby("Product ID").size()
    valid_products = product_order_counts[product_order_counts >= min_users_per_product].index
    orders_with_categories = orders_with_categories[orders_with_categories["Product ID"].isin(valid_products)]

    # Dividir en conjunto de entrenamiento y prueba
    train_df = orders_with_categories.sample(frac=0.8, random_state=42)
    test_df = orders_with_categories.drop(train_df.index)

    # Análisis del dataset
    print(f"Total de órdenes después del filtrado: {len(orders_with_categories)}")
    print(f"Clientes únicos después del filtrado: {orders_with_categories['Customer ID'].nunique()}")
    print(f"Productos únicos después del filtrado: {orders_with_categories['Product ID'].nunique()}")
    print(f"Órdenes por cliente (distribución):")
    print(train_df.groupby("Customer ID").size().describe())

    return train_df, test_df

# 2. Crear matriz de usuario-producto
def create_user_product_matrix(orders_with_categories):
    user_encoder = LabelEncoder()
    product_encoder = LabelEncoder()

    orders_with_categories["User Index"] = user_encoder.fit_transform(orders_with_categories["Customer ID"])
    orders_with_categories["Product Index"] = product_encoder.fit_transform(orders_with_categories["Product ID"])

    num_users = orders_with_categories["User Index"].nunique()
    num_products = orders_with_categories["Product Index"].nunique()
    matrix = np.zeros((num_users, num_products))

    for _, row in orders_with_categories.iterrows():
        matrix[row["User Index"], row["Product Index"]] += 1

    # Análisis de la matriz usuario-producto
    density = np.count_nonzero(matrix) / (matrix.shape[0] * matrix.shape[1])
    print(f"Densidad de interacciones: {density:.4f}")
    print(f"Tamaño de la matriz: {matrix.shape}")
    print(f"Interacciones no nulas: {np.count_nonzero(matrix)}")

    return matrix, user_encoder, product_encoder, orders_with_categories

# 3. Recomendar productos usando similitud de coseno
def recommend_products(user_id, matrix, user_encoder, product_encoder, orders_with_categories, num_recommendations=5):
    if user_id not in user_encoder.classes_:
        print(f"El usuario {user_id} no tiene datos suficientes para generar recomendaciones.")
        return [], []

    user_index = user_encoder.transform([user_id])[0]
    user_similarities = cosine_similarity(matrix)
    similar_users = np.argsort(-user_similarities[user_index])

    user_categories = orders_with_categories[orders_with_categories["Customer ID"] == user_id]["Category"].unique()
    candidate_products = orders_with_categories[orders_with_categories["Category"].isin(user_categories)]["Product Index"].unique()

    # Excluir productos ya comprados por el usuario
    purchased_products = orders_with_categories[orders_with_categories["Customer ID"] == user_id]["Product Index"].unique()
    candidate_products = [product for product in candidate_products if product not in purchased_products]

    # Recomendar productos basados en usuarios similares
    scores = matrix[similar_users[1:], :].sum(axis=0)
    max_score = scores.max() if scores.max() > 0 else 1
    normalized_scores = [(i, scores[i] / max_score) for i in candidate_products if scores[i] > 0]
    normalized_scores = sorted(normalized_scores, key=lambda x: x[1], reverse=True)

    product_mapping = orders_with_categories[["Product Index", "Product ID", "Name"]].drop_duplicates()
    product_mapping = product_mapping.set_index("Product Index")

    recommended_products = [
        (product_mapping.loc[product[0], "Product ID"], product_mapping.loc[product[0], "Name"], round(product[1], 2))
        for product in normalized_scores[:num_recommendations]
    ]

    user_purchases = orders_with_categories[orders_with_categories["Customer ID"] == user_id][["Product ID", "Name", "Category"]].drop_duplicates()

    return recommended_products, user_purchases

# 4. Guardar el modelo completo
def save_model(matrix, user_encoder, product_encoder, orders_with_categories, file_path="recommendation_model.pkl"):
    model_data = {
        "matrix": matrix,
        "user_encoder": user_encoder,
        "product_encoder": product_encoder,
        "orders_with_categories": orders_with_categories,
    }
    with open(file_path, "wb") as file:
        pickle.dump(model_data, file)
    print(f"Modelo guardado en {file_path}")

# Ejecutar el sistema de recomendación
if __name__ == "__main__":
    # Filtrar usuarios con pocas órdenes y productos con pocas compras
    train_df, test_df = extract_data(min_orders_per_user=5, min_users_per_product=5)
    matrix, user_encoder, product_encoder, orders_with_categories = create_user_product_matrix(train_df)

    # Guardar el modelo al final del entrenamiento
    save_model(matrix, user_encoder, product_encoder, orders_with_categories)


Total de órdenes después del filtrado: 134154
Clientes únicos después del filtrado: 1000
Productos únicos después del filtrado: 2193
Órdenes por cliente (distribución):
count    1000.000000
mean      107.323000
std        35.341869
min         6.000000
25%        95.750000
50%       114.000000
75%       129.000000
max       195.000000
dtype: float64
Densidad de interacciones: 0.0290
Tamaño de la matriz: (1000, 2193)
Interacciones no nulas: 63534
Modelo guardado en recommendation_model.pkl
